In [1]:
import numpy as np
import pandas as pd
import xarray as xr

### Open sample data
No QC is being used for this tutorial

In [2]:
file = '/scratch1/NCEPDEV/da/Kevin.Dougherty/plotting/data/gfs.a00f.diag.PT6H.amsua_n19.2020-12-16T21:00:00Z.PT6H.nc4'

ds = xr.open_dataset(file)
validvars = [var for var in list(ds.keys()) if var.endswith(('@hofx', '@EffectiveQC',
                                                             '@EffectiveError', '@ObsBias',
                                                             '@GsiHofXBc', '@PreQC',
                                                             '@GsiFinalObsError', 'latitude@MetaData',
                                                             'longitude@MetaData'))]

df = ds[validvars].to_dataframe()

df

,brightness_temperature_1@PreQC,brightness_temperature_1@EffectiveQC,brightness_temperature_10@PreQC,brightness_temperature_10@EffectiveQC,brightness_temperature_11@PreQC,brightness_temperature_11@EffectiveQC,brightness_temperature_12@PreQC,brightness_temperature_12@EffectiveQC,brightness_temperature_13@PreQC,brightness_temperature_13@EffectiveQC,...,brightness_temperature_8@ObsBias,brightness_temperature_8@EffectiveError,brightness_temperature_8@hofx,brightness_temperature_9@GsiFinalObsError,brightness_temperature_9@GsiHofXBc,brightness_temperature_9@ObsBias,brightness_temperature_9@EffectiveError,brightness_temperature_9@hofx,latitude@MetaData,longitude@MetaData
nlocs,,,,,,,,,,,,,,,,,,,,,
0,51.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.182202,NaN,214.643051,0.250007,217.742355,-0.348505,0.250006,217.728455,-86.515503,11.392600
1,51.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.400047,NaN,215.612198,0.250006,216.682724,-0.698139,0.250006,216.677582,-76.463997,18.673901
2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.747304,NaN,216.595673,0.250007,217.726151,-0.961956,0.250006,217.723618,-71.291801,24.991100
3,51.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.634131,NaN,217.469971,0.250006,217.875015,-0.593663,0.250005,217.863052,-65.656403,24.236700
4,51.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.971210,NaN,212.792786,0.250006,214.863403,-0.232083,0.250005,214.851654,-82.722801,37.968102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9092,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.374577,NaN,212.142212,0.250008,203.668884,-1.285095,0.250007,203.666748,-16.346300,311.379608
9093,51.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.468785,NaN,213.569504,0.250005,215.214325,-0.256490,0.250005,215.202286,-81.319801,348.330902
9094,51.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.393204,NaN,217.412979,0.250006,220.348099,-0.664964,0.250006,220.322708,-74.278503,339.351013


### Spatial Plots

In [3]:
from diagnostics import spatial


metadata = {'title': 'title',
            'cycle': 'cycle',
            'outfig': 'spatial_1',
            'cmap': 'viridis',
            'label': 'label'}

# Spatial of specific variable
spatial(df, metadata, variable='brightness_temperature_1@hofx')

# Spatial for just coverage
metadata = {'title': 'title',
            'cycle': 'cycle',
            'outfig': 'spatial_2',
            'cmap': 'black',
            'label': 'label'}

spatial(df, metadata)

/scratch1/NCEPDEV/da/Kevin.Dougherty/anaconda3/envs/UFO/lib/python3.8/site-packages/shapefile.py:385: UserWarning: Shapefile shape has invalid polygon: no exterior rings found (must have clockwise orientation); interpreting holes as exteriors.
  warnings.warn('Shapefile shape has invalid polygon: no exterior rings found (must have clockwise orientation); interpreting holes as exteriors.')


### Binned Spatial Plot

In [4]:
from diagnostics import bin_df, spatial_binned

# bin data first
binned_data = bin_df(df, variable='brightness_temperature_1@hofx', dlat=5, dlon=5)

# plot binned data spatially
metadata = {'title': 'title',
            'cycle': 'cycle',
            'outfig': 'spatial_binned_1',
            'cmap': 'viridis',
            'vmax': 300,
            'vmin': 200,
            'label': 'label'}

spatial_binned(binned_data, metadata)

# grab binned data and the count data as well
binned_data, count = bin_df(df, variable='brightness_temperature_1@hofx', dlat=5, dlon=5, return_count=True)

# plot obs count per bin
metadata = {'title': 'title',
            'cycle': 'cycle',
            'outfig': 'spatial_binned_2',
            'cmap': 'viridis',
            'vmax': 20,
            'vmin': 0,
            'label': 'label'}

spatial_binned(count, metadata)

### Scatter Plots

In [5]:
from diagnostics import scatter

metadata = {'title': 'title',
            'xlabel': 'xlabel',
            'ylabel': 'ylabel',
            'cycle': 'cycle',
            'outfig': 'scatter'
           }

# Simple Scatter Plot w/ Regression line
scatter(df['brightness_temperature_1@GsiHofXBc'], df['brightness_temperature_1@hofx'], metadata=metadata)

# Density Scatter Plot w/ Regression Line
scatter(df['brightness_temperature_1@GsiHofXBc'], df['brightness_temperature_1@hofx'], metadata=metadata, density=True)

### Line Plot

In [6]:
from diagnostics import lineplot

metadata = {'title': 'title',
            'cycle': 'cycle',
            'xlabel': 'location',
            'ylabel': 'UFO H(x) - GSI H(x)',
            'outfig': 'line',
            'linestyle': '-',
            'linewidth': 1,
            'color': 'red',
            'label': 'UFO - GSI'}

lineplot(df['brightness_temperature_1@hofx'].index,
         df[f'brightness_temperature_1@hofx']-df[f'brightness_temperature_1@GsiHofXBc'],
         metadata)